In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import black


def print_res(old_d: dict, result, just_ans=True):
    d = old_d.copy()
    d["tools"] = eval(d["tools"])
    if just_ans:
        del d["tools"]
    d["answers"] = eval(d["answers"])
    print("-" * 12, end="\n\n\n")
    print(black.format_str(str(d), mode=black.Mode()))
    print("-" * 12, end="\n\n\n")
    print(result)

In [20]:
import re
from parser import parse


def handle_response(response):
    pattern = re.compile(r"(?s)```shonqol\s*([\s\S]*?)\s*```")
    matches = pattern.findall(response)

    # for code_block in matches:
    return parse(matches[0])

In [22]:
def calc_hit_fn_arg(answers, extracted):
    hit_fn = 0
    hit_arg = 0
    tot_args = 0

    # Filter out non-dictionary entries and entries missing keys
    valid_answers = [
        a for a in answers if isinstance(a, dict) and "name" in a and "arguments" in a
    ]
    valid_extracted = [
        e for e in extracted if isinstance(e, dict) and "name" in e and "arguments" in e
    ]

    # Create a lookup for extracted functions by name
    extracted_lookup = [(e["name"], e) for e in valid_extracted]
    fn_names_set= set([e["name"] for e in valid_extracted])
    # Create a lookup for extracted functions by name mapping to a list of their 'arguments' lists
    extracted_lookup = {}
    for e in valid_extracted:
        extracted_lookup.setdefault(e["name"], []).append(e["arguments"])

    for a in valid_answers:
        fn_name = a["name"]
        if fn_name in extracted_lookup:
            hit_fn += 1
            ans_args = list(a["arguments"].values())
            tot_args += len(ans_args)
            for aarg in ans_args:
                # Check across all extracted function calls for this name, order doesn't matter
                if any(aarg in ext_args for ext_args in extracted_lookup[fn_name]):
                    hit_arg += 1

    print("hit_fn", hit_fn, "out of", len(valid_answers))
    print("hit_arg", hit_arg, "out of", tot_args)
    return (hit_fn,len(valid_answers)), (hit_arg, tot_args)


# Example data:
extracted = [
    {"name": "get_id", "arguments": ["Pulp Fiction"]},
    {"name": "search_torrents", "arguments": ["Pulp Fiction torrents"]},
    "Pulp Fiction torrents",
]


answers = [
    {"name": "get_id", "arguments": {"q": "Pulp Fiction"}},
    {
        "name": "search_torrents",
        "arguments": {"keywords": "Pulp Fiction", "quantity": 10},
    },
]

calc_hit_fn_arg(answers, extracted)

hit_fn 2 out of 2
hit_arg 1 out of 3


((2, 2), (1, 3))

In [6]:
import pandas as pd

# df = pd.read_json("hf://datasets/Salesforce/xlam-function-calling-60k/xlam_function_calling_60k.json")
df = pd.read_json("./xlam_function_calling_60k.json")

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from prompts import main_prompt

prompt = ChatPromptTemplate.from_messages(
    [("system", main_prompt), ("user", "{query}")]
)
llm = prompt | ChatOpenAI(model="gpt-4o-mini", temperature=0.7) | StrOutputParser()

In [10]:
d = df.iloc[6].to_dict()
result = llm.invoke(d)
print_res(d, result)

------------


{
    "id": 6,
    "query": "Identify the ID of 'Pulp Fiction' and search for torrents related to it.",
    "answers": [
        {"name": "get_id", "arguments": {"q": "Pulp Fiction"}},
        {
            "name": "search_torrents",
            "arguments": {"keywords": "Pulp Fiction", "quantity": 10},
        },
    ],
}

------------


```shonqol
movie_id = get_id("Pulp Fiction");
search_torrents(concat("Pulp Fiction", " torrent"), 40);
```


In [11]:
import json


extracted_functions = handle_response(result)
print("*" * 20)
print((extracted_functions))
print("*" * 20)
print(d["answers"])
print("*" * 20)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "get_id",
        "arguments": [
            "Pulp Fiction"
        ]
    },
    {
        "name": "search_torrents",
        "arguments": [
            "Pulp Fiction torrent",
            40
        ]
    },
    "Pulp Fiction torrent"
]
********************
[{'name': 'get_id', 'arguments': ['Pulp Fiction']}, {'name': 'search_torrents', 'arguments': ['Pulp Fiction torrent', 40]}, 'Pulp Fiction torrent']
********************
[{"name": "get_id", "arguments": {"q": "Pulp Fiction"}}, {"name": "search_torrents", "arguments": {"keywords": "Pulp Fiction", "quantity": 10}}]
********************
hit_fn 2 out of 2
hit_arg 1 out of 3


line 2:55 extraneous input ';' expecting {<EOF>, Identifier}


(2, 1)

In [41]:
d = df.iloc[9].to_dict()
result = llm.invoke(d)
print_res(d, result)

------------


{
    "id": 9,
    "query": "Is 'https://www.apple.com' available in the Wayback Machine on September 9, 2015?",
    "answers": [
        {
            "name": "availability",
            "arguments": {"url": "https://www.apple.com", "timestamp": "20150909"},
        }
    ],
}

------------


```shonqol
availability("https://www.apple.com", "20150909");
```


In [43]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "availability",
        "arguments": [
            "https://www.apple.com",
            "20150909"
        ]
    }
]
hit_fn 1 out of 1
hit_arg 2 out of 2


line 1:49 extraneous input ';' expecting {<EOF>, Identifier}


(1, 2)

In [44]:
d = df.iloc[103].to_dict()
result = llm.invoke(d)
print_res(d, result, just_ans=False)

------------


{
    "id": 103,
    "query": "What is the recommended daily calorie intake and macronutrient distribution for a 30-year-old male, weighing 75 kg, 180 cm tall, moderately active, aiming to maintain weight?",
    "answers": [
        {
            "name": "calculate_calorie_intake",
            "arguments": {
                "weight_kg": 75,
                "height_cm": 180,
                "age": 30,
                "sex": "male",
                "activity_level": 3,
                "goal": "maintain",
            },
        }
    ],
    "tools": [
        {
            "name": "calculate_calorie_intake",
            "description": "Calculates the recommended daily calorie intake and macronutrient distribution based on personal characteristics and goals.",
            "parameters": {
                "weight_kg": {
                    "description": "Body weight in kilograms.",
                    "type": "float",
                },
                "height_cm": {"descript

In [45]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "calculate_calorie_intake",
        "arguments": [
            75,
            180,
            30,
            "male",
            3,
            "maintain"
        ]
    }
]
hit_fn 1 out of 1
hit_arg 6 out of 6


line 1:60 extraneous input ';' expecting {<EOF>, Identifier}


(1, 6)

In [46]:
d = df.iloc[586].to_dict()
result = llm.invoke(d)
print_res(d, result, just_ans=True)

------------


{
    "id": 586,
    "query": "I'm curious about the neuronal activity rate when the synaptic input rate is 100 per second, with a synaptic weight of 0.5 and a decay constant of 0.05. Also, could you check if 17 is a prime number and solve the quadratic equation 2x^2 - 3x + 1 = 0?",
    "answers": [
        {
            "name": "neuronal_activity_rate",
            "arguments": {
                "synaptic_input_rate": 100,
                "synaptic_weight": 0.5,
                "decay_constant": 0.05,
            },
        },
        {"name": "is_prime", "arguments": {"num": 17}},
        {"name": "solve_quadratic", "arguments": {"a": 2, "b": -3, "c": 1}},
    ],
}

------------


```shonqol
neuronal_activity = neuronal_activity_rate(100, 0.5, 0.05);
is_seventeen_prime = is_prime(17);
quadratic_roots = solve_quadratic(2, -3, 1);
```


In [50]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "neuronal_activity_rate",
        "arguments": [
            100,
            0.5,
            0.05
        ]
    },
    {
        "name": "is_prime",
        "arguments": [
            17
        ]
    },
    {
        "name": "solve_quadratic",
        "arguments": [
            2,
            -3,
            1
        ]
    }
]
hit_fn 3 out of 3
hit_arg 7 out of 7


(3, 7)

In [12]:
d = df.iloc[998].to_dict()
result = llm.invoke(d)
print_res(
    d,
    result,
)

------------


{
    "id": 998,
    "query": "List all tokens on the Ethereum network.",
    "answers": [{"name": "tokens", "arguments": {"network": "ethereum"}}],
}

------------


```shonqol
tokens("ethereum");
```


In [13]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "tokens",
        "arguments": [
            "ethereum"
        ]
    }
]
hit_fn 1 out of 1
hit_arg 1 out of 1


line 1:18 extraneous input ';' expecting {<EOF>, Identifier}


(1, 1)

In [17]:
import json

for _, row in df.iterrows():

    v = json.loads(str(row["answers"]))
    if not isinstance(v, list):
        continue
    if len(v) > 3:
        d = row.to_dict()
        
        break
result = llm.invoke(d)
print_res(d, result, just_ans=True)

------------


{
    "id": 107,
    "query": "I'm conducting a research study and need the men's marks for points values of 10, 20, 30, 40, and 50 to observe the trend in the lower range.",
    "answers": [
        {"name": "marks_men_points", "arguments": {"points": 10}},
        {"name": "marks_men_points", "arguments": {"points": 20}},
        {"name": "marks_men_points", "arguments": {"points": 30}},
        {"name": "marks_men_points", "arguments": {"points": 40}},
        {"name": "marks_men_points", "arguments": {"points": 50}},
    ],
}

------------


```shonqol
marks_10 = marks_men_points(10);
marks_20 = marks_men_points(20);
marks_30 = marks_men_points(30);
marks_40 = marks_men_points(40);
marks_50 = marks_men_points(50);
```


In [18]:
extracted_functions = handle_response(result)
calc_hit_fn_arg(json.loads( d["answers"]), extracted_functions)

Extracted function calls from AST:
[
    {
        "name": "marks_men_points",
        "arguments": [
            10
        ]
    },
    {
        "name": "marks_men_points",
        "arguments": [
            20
        ]
    },
    {
        "name": "marks_men_points",
        "arguments": [
            30
        ]
    },
    {
        "name": "marks_men_points",
        "arguments": [
            40
        ]
    },
    {
        "name": "marks_men_points",
        "arguments": [
            50
        ]
    }
]
hit_fn 5 out of 5
hit_arg 5 out of 5


(5, 5)

In [21]:
df.sample(1)

,id,query,answers,tools
18675,18675,Fetch the 5-day forecast for London's coordina...,"[{""name"": ""daily_forecast_5_days"", ""arguments""...","[{""name"": ""alerts_active_zone_zoneid"", ""descri..."


In [23]:
# Choose 10 random entries from the DataFrame
import os


sample_df = df.sample(10)
if not os.path.exists("outputs"):
    os.makedirs("outputs")
for idx, row in sample_df.iterrows():
    d = row.to_dict()
    # Call the language model as done in previous cells
    result = llm.invoke(d)
    print_res(d, result)
    
    # Process extracted functions and calculate hit metrics
    extracted_functions = handle_response(result)
    hit_result=calc_hit_fn_arg(json.loads(d["answers"]), extracted_functions)
    print("-" * 40)
    
    # Write d, result, and extracted_functions to a json file
    data_to_save = {
        'id':d['id'],
        "d": d,
        "result": result,
        "extracted_functions": extracted_functions,
        "hit_results":{ "hit_fn":hit_result[0], "hit_arg":hit_result[1]}
    }
    with open(f"outputs/output_{d['id']}.json", "w") as f:
        json.dump(data_to_save, f, indent=2)

------------


{
    "id": 57109,
    "query": "Get the latest coronavirus news for France.",
    "answers": [{"name": "getnews", "arguments": {"location": "FR"}}],
}

------------


```shonqol
getnews("FR");
```
hit_fn 1 out of 1
hit_arg 1 out of 1
----------------------------------------


line 1:13 extraneous input ';' expecting {<EOF>, Identifier}


------------


{
    "id": 57817,
    "query": "What is the duration between a note with a frequency of 440 Hz and a note with a frequency of 660 Hz, given a tempo of 60 beats per minute? Also, find the longest palindromic substring in the string 'racecar is racing'.",
    "answers": [
        {"name": "note_duration", "arguments": {"freq1": 440, "freq2": 660, "bpm": 60}},
        {
            "name": "find_longest_palindromic_substring",
            "arguments": {"s": "racecar is racing"},
        },
    ],
}

------------


```shonqol
duration = note_duration(440, 660, 60);
longest_palindrome = find_longest_palindromic_substring("racecar is racing");
```
hit_fn 2 out of 2
hit_arg 4 out of 4
----------------------------------------
------------


{
    "id": 29952,
    "query": "Fetch OpenGraph and Meta tags from the website of a popular news outlet.",
    "answers": [{"name": "og", "arguments": {"url": "https://www.bbc.com/"}}],
}

------------


```shonqol
og("https://www.bbc.com")

line 1:25 extraneous input ';' expecting {<EOF>, Identifier}


------------


{
    "id": 7800,
    "query": "Find the missing number in the list [0, 1, 2, 3, 5, 6, 7, 8, 9] which should contain numbers from 0 to 9.",
    "answers": [
        {
            "name": "find_missing_number",
            "arguments": {"nums": [0, 1, 2, 3, 5, 6, 7, 8, 9]},
        }
    ],
}

------------


```shonqol
missing_number = find_missing_number([0, 1, 2, 3, 5, 6, 7, 8, 9]);
```
hit_fn 1 out of 1
hit_arg 0 out of 1
----------------------------------------


...

line 1:27 extraneous input ';' expecting {<EOF>, Identifier}


------------


{
    "id": 20864,
    "query": "Are jobs 678901 and 234567 still processing?",
    "answers": [
        {"name": "get_job_status", "arguments": {"is_id": "678901"}},
        {"name": "get_job_status", "arguments": {"is_id": "234567"}},
    ],
}

------------


```shonqol
status1 = get_job_status("678901");
status2 = get_job_status("234567");

isProcessing1 = conditionalSelect(
    equal(status1, "processing"),
    "Job 678901 is still processing.",
    "Job 678901 is not processing."
);

isProcessing2 = conditionalSelect(
    equal(status2, "processing"),
    "Job 234567 is still processing.",
    "Job 234567 is not processing."
);

print(isProcessing1);
print(isProcessing2);
```
hit_fn 2 out of 2
hit_arg 2 out of 2
----------------------------------------


line 16:20 extraneous input ';' expecting {<EOF>, Identifier}
line 17:20 extraneous input ';' expecting {<EOF>, Identifier}


------------


{
    "id": 20142,
    "query": "List all holidays in the USA for the year 2023.",
    "answers": [
        {
            "name": "get_holidays_by_year",
            "arguments": {"year": 2023, "country_code": "US"},
        }
    ],
}

------------


```shonqol
get_holidays_by_year(2023, "US");
```
hit_fn 1 out of 1
hit_arg 2 out of 2
----------------------------------------


line 1:32 extraneous input ';' expecting {<EOF>, Identifier}
